In [1]:
# Import dependencies
import psycopg2
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import tensorflow as tf
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
from google.colab import files
uploaded = files.upload()

Saving All_Data.csv to All_Data.csv


In [3]:
# Pull in working dataset
all_data_df = pd.read_csv("All_Data.csv")
all_data_df.head()

,FIPS,Num_Per_100k,State,County,Percent_Smokers,Percent_Obese,Food_Environment_Index,Percent_Access_Exercise,Primary_Care_Physician_Rate,Mental_Health_Providers_Rate,Percent_Some_College,Percent_Unemployed,Income_Inequality_Ratio,Violent_Crime_Rate,Average_Daily_Air_Polution_Particle_Matter,Percent_Long_Commute_Drives_Alone,Percent_Rural
0,1001,107.0,Alabama,Autauga,19.0,36.0,7.1,71.0,45.0,13.0,61.9,5.3,4.4,266.0,10.8,41.0,42.0
1,1003,82.5,Alabama,Baldwin,17.0,29.0,7.9,69.0,73.0,89.0,63.8,5.4,4.6,217.0,9.6,40.0,42.3
2,1005,96.1,Alabama,Barbour,22.0,44.0,5.5,53.0,42.0,8.0,39.7,8.6,5.9,329.0,10.1,35.0,67.8
3,1007,112.3,Alabama,Bibb,20.0,38.0,7.6,49.0,53.0,9.0,49.8,6.6,4.2,147.0,10.5,49.0,68.4
4,1009,96.1,Alabama,Blount,20.0,36.0,8.5,32.0,21.0,9.0,53.9,5.5,4.1,212.0,11.3,60.0,90.0


In [4]:
# Check for null values
all_data_df.isnull().sum()

FIPS                                            0
Num_Per_100k                                    7
State                                           0
County                                          0
Percent_Smokers                                 0
Percent_Obese                                   0
Food_Environment_Index                         19
Percent_Access_Exercise                         6
Primary_Care_Physician_Rate                   134
Mental_Health_Providers_Rate                  264
Percent_Some_College                            0
Percent_Unemployed                              1
Income_Inequality_Ratio                         2
Violent_Crime_Rate                            176
Average_Daily_Air_Polution_Particle_Matter     34
Percent_Long_Commute_Drives_Alone               0
Percent_Rural                                   7
dtype: int64

In [5]:
# Drop null values
all_data_df = all_data_df.dropna(axis=0)

In [6]:
# Drop identifier columns
all_data_usable_df = all_data_df.drop(["FIPS", "State", "County"], 1)
all_data_usable_df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


,Num_Per_100k,Percent_Smokers,Percent_Obese,Food_Environment_Index,Percent_Access_Exercise,Primary_Care_Physician_Rate,Mental_Health_Providers_Rate,Percent_Some_College,Percent_Unemployed,Income_Inequality_Ratio,Violent_Crime_Rate,Average_Daily_Air_Polution_Particle_Matter,Percent_Long_Commute_Drives_Alone,Percent_Rural
0,107.0,19.0,36.0,7.1,71.0,45.0,13.0,61.9,5.3,4.4,266.0,10.8,41.0,42.0
1,82.5,17.0,29.0,7.9,69.0,73.0,89.0,63.8,5.4,4.6,217.0,9.6,40.0,42.3
2,96.1,22.0,44.0,5.5,53.0,42.0,8.0,39.7,8.6,5.9,329.0,10.1,35.0,67.8
3,112.3,20.0,38.0,7.6,49.0,53.0,9.0,49.8,6.6,4.2,147.0,10.5,49.0,68.4
4,96.1,20.0,36.0,8.5,32.0,21.0,9.0,53.9,5.5,4.1,212.0,11.3,60.0,90.0


Random Forest Regressor - all features

In [7]:
# https://stackabuse.com/random-forest-algorithm-with-python-and-scikit-learn/
# Divide data into attributes and labels

X = all_data_usable_df.iloc[:, 1:13].values
y = all_data_usable_df.iloc[:, 0].values

In [8]:
# Divide data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [9]:
# Feature Scaling
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [10]:
# Train random forest algorithm to solve this regression problem
regressor = RandomForestRegressor(n_estimators=20, random_state=0)
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 9.502098298676747
Mean Squared Error: 150.77297816635158
Root Mean Squared Error: 12.278964865425406


In [11]:
# Increase estimators to 50
regressor2 = RandomForestRegressor(n_estimators=50, random_state=0)
regressor2.fit(X_train, y_train)
y_pred2 = regressor2.predict(X_test)

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred2))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred2))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred2)))

Mean Absolute Error: 9.240952741020793
Mean Squared Error: 144.93051641587903
Root Mean Squared Error: 12.038709084278057


In [12]:
# Change estimators to 30
regressor3 = RandomForestRegressor(n_estimators=30, random_state=0)
regressor3.fit(X_train, y_train)
y_pred3 = regressor3.predict(X_test)

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred3))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred3))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred3)))

Mean Absolute Error: 9.387511027095147
Mean Squared Error: 149.0808244906532
Root Mean Squared Error: 12.209865867021358


Random Forest Regressor - health features

In [13]:
# Drop non-health feature columns
health_features_df = all_data_usable_df.drop(["Percent_Rural", "Percent_Long_Commute_Drives_Alone", "Average_Daily_Air_Polution_Particle_Matter", "Violent_Crime_Rate", "Income_Inequality_Ratio", "Percent_Unemployed", "Percent_Some_College"], 1)
health_features_df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


,Num_Per_100k,Percent_Smokers,Percent_Obese,Food_Environment_Index,Percent_Access_Exercise,Primary_Care_Physician_Rate,Mental_Health_Providers_Rate
0,107.0,19.0,36.0,7.1,71.0,45.0,13.0
1,82.5,17.0,29.0,7.9,69.0,73.0,89.0
2,96.1,22.0,44.0,5.5,53.0,42.0,8.0
3,112.3,20.0,38.0,7.6,49.0,53.0,9.0
4,96.1,20.0,36.0,8.5,32.0,21.0,9.0


In [14]:
# Divide into attributes and labels
X = all_data_usable_df.iloc[:, 1:7].values
y = all_data_usable_df.iloc[:, 0].values

In [15]:
# Divide data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [16]:
# Scale Features
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [17]:
# Train random forest algorithm to solve this regression problem
regressor = RandomForestRegressor(n_estimators=20, random_state=0)
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 10.390255198487711
Mean Squared Error: 178.18698993383742
Root Mean Squared Error: 13.348669968721133


In [18]:
# Increase estimators to 50
regressor2 = RandomForestRegressor(n_estimators=50, random_state=0)
regressor2.fit(X_train, y_train)
y_pred2 = regressor2.predict(X_test)

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred2))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred2))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred2)))

Mean Absolute Error: 10.262215500945178
Mean Squared Error: 172.17535106238185
Root Mean Squared Error: 13.121560542190927


In [19]:
# Change estimators to 30
regressor3 = RandomForestRegressor(n_estimators=30, random_state=0)
regressor3.fit(X_train, y_train)
y_pred3 = regressor3.predict(X_test)

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred3))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred3))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred3)))

Mean Absolute Error: 10.255551354757404
Mean Squared Error: 173.63839512707418
Root Mean Squared Error: 13.177192232303291


Random Forest Regressor - environment / economic features

In [20]:
ee_features_df = all_data_usable_df.drop(["Percent_Smokers", "Percent_Obese", "Food_Environment_Index", "Percent_Access_Exercise", "Primary_Care_Physician_Rate", "Mental_Health_Providers_Rate"], 1)
ee_features_df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


,Num_Per_100k,Percent_Some_College,Percent_Unemployed,Income_Inequality_Ratio,Violent_Crime_Rate,Average_Daily_Air_Polution_Particle_Matter,Percent_Long_Commute_Drives_Alone,Percent_Rural
0,107.0,61.9,5.3,4.4,266.0,10.8,41.0,42.0
1,82.5,63.8,5.4,4.6,217.0,9.6,40.0,42.3
2,96.1,39.7,8.6,5.9,329.0,10.1,35.0,67.8
3,112.3,49.8,6.6,4.2,147.0,10.5,49.0,68.4
4,96.1,53.9,5.5,4.1,212.0,11.3,60.0,90.0


In [21]:
# Divide into attributes and labels
X = all_data_usable_df.iloc[:, 1:6].values
y = all_data_usable_df.iloc[:, 0].values

In [22]:
# Divide data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [23]:
# Scale Features
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [24]:
# Train random forest algorithm to solve this regression problem
regressor = RandomForestRegressor(n_estimators=20, random_state=0)
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 10.528317580340264
Mean Squared Error: 182.19479536862002
Root Mean Squared Error: 13.49795522916786


In [25]:
# Increase estimators to 50
regressor2 = RandomForestRegressor(n_estimators=50, random_state=0)
regressor2.fit(X_train, y_train)
y_pred2 = regressor2.predict(X_test)

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred2))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred2))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred2)))

Mean Absolute Error: 10.329909262759925
Mean Squared Error: 174.4235131115312
Root Mean Squared Error: 13.20694942488731


In [26]:
# Change estimators to 30
regressor3 = RandomForestRegressor(n_estimators=30, random_state=0)
regressor3.fit(X_train, y_train)
y_pred3 = regressor3.predict(X_test)

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred3))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred3))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred3)))

Mean Absolute Error: 10.43471329552615
Mean Squared Error: 178.69157187565636
Root Mean Squared Error: 13.367556690572005


Random Forest Regressor - features of importance